In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(42)

Supongamos que nos dan esta serie de puntos

In [ ]:
N_SAMPLES = 20
x = np.linspace(0,1,num=N_SAMPLES).reshape(-1,1)
t =  4 * x+ np.sin(x*6)+ 0.5*np.random.randn(N_SAMPLES,1)

In [ ]:
plt.scatter(x,t)
plt.xlabel('x')
plt.ylabel('t')
plt.show()

Por supuesto, antes de seguir dividimos en entrenamiento y testeo

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, t_train, t_test = train_test_split(x, t, random_state=42)


In [ ]:
plt.scatter(x_train,t_train, c='red', label='train')
plt.scatter(x_test,t_test, c='blue', label='test')
plt.legend()
plt.xlabel('x')
plt.ylabel('y')
plt.show()

Como vimos, la idea de Regresion Lineal es modelar

$$y(x,w)=w_{0}+w_{1}x$$

De manera tal que 

$$t\sim \mathcal{N}(y(x,w),\sigma^{2})$$

Con alguna incerteza que no nos importa demasiado hoy.

Aca tenemos la gran ventaja de que sabemos la verdadera funcion que sigue el target. En la vida real, casi nunca sabemos la verdadera funcion.

In [ ]:
def y_true(x):
  return  4 * x+ np.sin(x*6)

In [ ]:
xvals=np.linspace(0,1,num=50).reshape(-1,1)
plt.scatter(x_train,t_train, c='red', label='train')
plt.plot(xvals,y_true(xvals),color='black',label='Funcion verdadera')
plt.legend()
plt.xlabel('x')
plt.ylabel('t')
plt.show()

Veamos los shapes que tenemos

In [ ]:
print(x_train.shape,t_train.shape)

# Regresion Lineal a mano

Si queremos obtener los estimador de maxima verosimilitud de los pesos $w_{0}$, $w_{1}$, tenemos que resolver las ecuaciones normales

$$w_{ML}=(\Phi^{T}\Phi)^{-1}\Phi^{T}t$$

Donde $w_{ML}=(w_{0,ML},w_{1,ML})^{T}$, $t$ el vector de target y $\Phi$ es la matriz de diseño. En cada columna, tiene una de las funciones de base. En este caso, 1 y x. En cada fila, tiene una medicion

Si tenemos M funciones de base y N mediciones, 

$w_{ML}$ es una matriz de

$\Phi$ es una matriz de 

$t$ es una matriz de

In [ ]:
Phi_train=np.vstack([np.ones(len(x_train)),x_train[:,0]]).T # agrupo 1s y xs para cada medicion

Que shape tiene que tener?

In [ ]:
Phi_train.shape

Las ecuaciones normales tienen una inversa ahi metida. Eso no es numericamente muy deseable. Conviene resolver la ecuacion

$$(\Phi^{T}\Phi)w_{ML}=\Phi^{T}t$$

y utilizar `np.linalg.solve(A,b)` que obtiene el vector $X$ solucion de la ecuacion

$$Ax=b$$



In [ ]:
A=
b=
wML=

Veamos la solucion

In [ ]:
wML

Podemos graficar esta solucion facilmente

In [ ]:
def y_ML(x,wML):
  return wML[0]+wML[1]*x

In [ ]:
xvals=np.linspace(0,1,num=50).reshape(-1,1)
plt.scatter(x_train,t_train, c='red', label='train')
plt.plot(xvals,y_true(xvals),color='black',label='Funcion verdadera')
plt.plot(xvals,y_ML(xvals,wML),color='black',linestyle='dotted',label='Maxima verosimilitud')
plt.legend()
plt.xlabel('x')
plt.ylabel('t')
plt.show()

Obviamente, no funciona muy bien...

## Ejercicio

Resuelvan las ecuaciones normales, pero ahora modelando la funcion como

$$y(x,w)=w_{0}+\sum_{i=1}^{5}w_{i}x^{i}$$

# Hola `sklearn`!

Como ya vimos antes, `sklearn` resuelve las ecuaciones normales por nosotros utilizando `LinearRegression`

In [ ]:
from sklearn.linear_model import LinearRegression

Veamos los parametros que tenemos:

In [ ]:
LinearRegression??

La opcion con la que hay que tener mas cuidado ahora mismo en `fit_intercept`. Por defecto, esta como verdadera. 

In [ ]:
lr=LinearRegression(fit_intercept=True)
lr.fit(x_train,t_train) #features, target

Podemos obtener tanto los coeficientes como el intercept

In [ ]:
print(lr.intercept_,lr.coef_)

Noten que tenemos 1 solo coeficiente. El $w_{0}$ es el intercept.

Supongamos que no nos damos cuenta y le damos la matriz $\Phi$

In [ ]:
lr=LinearRegression(fit_intercept=True)
lr.fit(Phi_train,t_train) #features, target

In [ ]:
print(lr.intercept_,lr.coef_)

Aparece el coeficiente que deberia ser el `intercept` como cero. 

Hay que tener cuidado con esto, en particular cuando queremos aprovechar `sklearn` para hacer regresion polinomial

In [ ]:
from sklearn.preprocessing import PolynomialFeatures # utilizo esto para convertir mi X es a la base que quiero

In [ ]:
PolynomialFeatures?

In [ ]:
PolynomialFeatures(degree=1).fit_transform(x_train)

In [ ]:
PolynomialFeatures(degree=1,include_bias=False).fit_transform(x_train)

Tenemos dos opciones: `PolynomialFeatures` con `include_bias=True` y `LinearRegression` con `fit_intercept=False` o el opuesto 

Podemos combinar ambos utilizando el `Pipeline`

In [ ]:
from sklearn.pipeline import Pipeline
Pipeline?

In [ ]:
degree=5
modelo=Pipeline([['poly',PolynomialFeatures(degree=degree,include_bias=True)],['regressor',LinearRegression(fit_intercept=False)]])

In [ ]:
modelo.fit(x_train,t_train)

Podemos ver los coeficientes con

In [ ]:
modelo['regressor'].coef_

Y verificar que no hay intercept

In [ ]:
modelo['regressor'].intercept_

## Ejercicios:

1.   Utilicen este Pipeline para loopear sobre grados, con que se encuentran?
2.   Utilicen las funciones de base mas adecuadas: [1,x,sin(6x)]. Que encuentra?